# Analysis of Surface Fields

`mom6_tools.MOM6grid` returns an object with MOM6 grid data.

`mom6_tools.latlon_analysis` has a collection of tools used to perform spatial analysis  (e.g., time averages and spatial mean).

The goal of this notebook is the following:

1) server as an example of how to post-process CESM/MOM6 output;

2) create time averages of surface fields;

3) create time-series of globally-averaged surface fields;

4) compare model results vs. observations (TODO);

5) compare model results vs. another model results (TODO).

In [1]:
%matplotlib inline
from mom6_tools.MOM6grid import MOM6grid
from mom6_tools.DiagsCase import DiagsCase
from mom6_tools.latlon_analysis import time_mean_latlon
import matplotlib.pyplot as plt
import warnings, yaml
warnings.filterwarnings("ignore")

Basemap module not found. Some regional plots may not function properly


In [2]:
# Read in the yaml file
diag_config_yml_path = "/glade/u/home/gmarques/libs/mom6-tools/docs/source/examples/diag_config.yml"
diag_config_yml = yaml.load(open(diag_config_yml_path,'r'), Loader=yaml.Loader)

In [3]:
# Create the case instance
dcase = DiagsCase(diag_config_yml['Case'])
RUNDIR = dcase.get_value('RUNDIR')
print('Run directory is:', RUNDIR)
print('Casename is:', dcase.casename)

Run directory is: /glade/scratch/gmarques/g.c2b6.GJRA.TL319_t061.long_JRA_mct.076/run
Casename is: g.c2b6.GJRA.TL319_t061.long_JRA_mct.076


In [4]:
# The following parameters must be set accordingly
######################################################
# Path to the run directory
path = "/glade/scratch/gmarques/"+dcase.casename+"/run/"
# name of ocean static file
static = dcase.casename+'.mom6.static.nc'
# name of forcing file
surface = dcase.casename+'.mom6.sfc_00*.nc'
# initial and final years for computing time mean
year_start = 1
year_end = 85
# array with 2D forcing variables to be processed
variables = ['SST','SSS','SSH']
# Put your name and email address below
author = 'Gustavo Marques (gmarques@ucar.edu)'
######################################################
# create an empty class object
class args:
  pass

args.infile = path+surface
# set avg dates
avg = diag_config_yml['Avg']
args.start_date = avg['start_date']
args.end_date = avg['end_date']
args.case_name = dcase.casename
args.variables = variables
args.savefigs = False
args.time_series = True
args.nw = 6 # requesting 6 workers

In [5]:
# load mom6 grid
grd = MOM6grid(path+static)
# plot time averages. If variables is NOT specified, all 2D variables will be plotted.
#time_mean_latlon_plot(args,grd, variables=args.variables)
time_mean_latlon(args,grd)

MOM6 grid successfully loaded... 

Requesting 6 workers... 

/proxy/38817/status


UnboundLocalError: local variable 'units' referenced before assignment